In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.simplefilter("ignore")

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [3]:
data=pd.read_excel("Concrete_Data.xlsx")
data.head()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),concrete
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Cement (component 1)(kg in a m^3 mixture)              1030 non-null   float64
 1   Blast Furnace Slag (component 2)(kg in a m^3 mixture)  1030 non-null   float64
 2   Fly Ash (component 3)(kg in a m^3 mixture)             1030 non-null   float64
 3   Water  (component 4)(kg in a m^3 mixture)              1030 non-null   float64
 4   Superplasticizer (component 5)(kg in a m^3 mixture)    1030 non-null   float64
 5   Coarse Aggregate  (component 6)(kg in a m^3 mixture)   1030 non-null   float64
 6   Fine Aggregate (component 7)(kg in a m^3 mixture)      1030 non-null   float64
 7   Age (day)                                              1030 non-null   int64  
 8   concrete                                         

In [5]:
#data.drop(columns=["Fine Aggregate (component 7)(kg in a m^3 mixture)"],inplace=True)

In [6]:
def plot_boxplot(data,ft):
    data.boxplot(column=[ft])
    plt.grid(False)
    plt.show()

In [7]:
def outliers(data,ft):
    Q1=data[ft].quantile(0.25)
    Q3=data[ft].quantile(0.75)
    IQR=Q3-Q1
    lower_bound=Q1-1.5*IQR
    upper_bound=Q3+1.5*IQR

    ls=data.index[ (data[ft] < lower_bound) | (data[ft]> upper_bound) ]

    return ls

In [8]:
index_list=[]
for feature in ["Cement (component 1)(kg in a m^3 mixture)","Blast Furnace Slag (component 2)(kg in a m^3 mixture)","Fly Ash (component 3)(kg in a m^3 mixture)","Water  (component 4)(kg in a m^3 mixture)","Superplasticizer (component 5)(kg in a m^3 mixture)","Coarse Aggregate  (component 6)(kg in a m^3 mixture)","Age (day)","concrete"]:
    index_list.extend(outliers(data,feature))
index_list

[553,
 559,
 224,
 225,
 226,
 227,
 228,
 862,
 873,
 936,
 1019,
 76,
 79,
 99,
 102,
 122,
 125,
 145,
 148,
 168,
 171,
 2,
 3,
 4,
 6,
 12,
 17,
 19,
 20,
 23,
 24,
 25,
 26,
 27,
 30,
 31,
 32,
 33,
 34,
 35,
 39,
 41,
 42,
 43,
 47,
 50,
 51,
 56,
 58,
 60,
 61,
 63,
 64,
 65,
 66,
 603,
 604,
 609,
 610,
 615,
 616,
 619,
 620,
 621,
 622,
 755,
 756,
 762,
 768,
 769,
 791,
 792,
 794,
 795,
 797,
 798,
 813,
 814,
 820,
 823,
 0,
 153,
 181,
 381]

In [9]:
def remove(data,ls):
    ls=sorted(set(ls))
    data=data.drop(ls)
    return data

In [10]:
data_cleaned=remove(data,index_list)

In [11]:
data_cleaned

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),concrete
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
5,266.0,114.0,0.0,228.0,0.0,932.0,670.0,90,47.029847
7,380.0,95.0,0.0,228.0,0.0,932.0,594.0,28,36.447770
8,266.0,114.0,0.0,228.0,0.0,932.0,670.0,28,45.854291
9,475.0,0.0,0.0,228.0,0.0,932.0,594.0,28,39.289790
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.284354
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.178794
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.696601
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.768036


In [12]:
data_cleaned.isnull().sum()

Cement (component 1)(kg in a m^3 mixture)                0
Blast Furnace Slag (component 2)(kg in a m^3 mixture)    0
Fly Ash (component 3)(kg in a m^3 mixture)               0
Water  (component 4)(kg in a m^3 mixture)                0
Superplasticizer (component 5)(kg in a m^3 mixture)      0
Coarse Aggregate  (component 6)(kg in a m^3 mixture)     0
Fine Aggregate (component 7)(kg in a m^3 mixture)        0
Age (day)                                                0
concrete                                                 0
dtype: int64

In [31]:
data_cleaned.skew()

Cement (component 1)(kg in a m^3 mixture)                0.510539
Blast Furnace Slag (component 2)(kg in a m^3 mixture)    0.760281
Fly Ash (component 3)(kg in a m^3 mixture)               0.429420
Water  (component 4)(kg in a m^3 mixture)               -0.007469
Superplasticizer (component 5)(kg in a m^3 mixture)      0.369544
Coarse Aggregate  (component 6)(kg in a m^3 mixture)    -0.101502
Fine Aggregate (component 7)(kg in a m^3 mixture)       -0.193862
Age (day)                                                1.236443
concrete                                                 0.446094
dtype: float64

In [33]:
from scipy.stats import boxcox
data_cleaned["Age (day)"],a=boxcox(data_cleaned["Age (day)"])

In [34]:
X=data_cleaned.drop("concrete",axis=1)
y=data_cleaned["concrete"]

In [35]:

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=9)

from sklearn.linear_model import LinearRegression
model1=LinearRegression()
model1.fit(X_train,y_train)

from sklearn.model_selection import cross_val_score
print("Intercept:",model1.intercept_)
print("coefficients:",model1.coef_)

train_predictions=model1.predict(X_train)
test_predictions=model1.predict(X_test)

print("Train R2:",model1.score(X_train,y_train))
print("Test R2:",model1.score(X_test,y_test))
print("cross validation score:",cross_val_score(model1,X,y,cv=5).mean())

Intercept: -33.088552762208
coefficients: [ 0.12258114  0.09785149  0.0707239  -0.1618712   0.15096923  0.01329407
  0.02062633  4.78622933]
Train R2: 0.8425946823635366
Test R2: 0.8380756173518644
cross validation score: 0.7452405573031636


In [36]:
import statsmodels.formula.api as smf
model2=smf.ols("y~X",data=data_cleaned).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.843
Model:                            OLS   Adj. R-squared:                  0.841
Method:                 Least Squares   F-statistic:                     627.2
Date:                Wed, 24 Apr 2024   Prob (F-statistic):               0.00
Time:                        00:18:26   Log-Likelihood:                -3125.6
No. Observations:                 946   AIC:                             6269.
Df Residuals:                     937   BIC:                             6313.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -21.5537     18.537     -1.163      0.245     -57.932      14.825
X[0]           0.1229      0.006     21.322      0.000       0.112       0.134
X[1]           0.0974      0.007     14.005      0.000       0.084       0.111
X[2]           0.0722      0.009      8.491      0.000       0.056       0.089
X[3]          -0.2005      0.029     -6.962      0.000      -0.257      -0.144
X[4]           0.0621      0.070      0.889      0.374      -0.075       0.199
X[5]           0.0125      0.006      1.934      0.053      -0.000       0.025
X[6]           0.0160      0.007      2.164      0.031       0.001       0.030
X[7]           4.8359      0.104     46.282      0.000       4.631       5.041
==============================================================================
Omnibus:                       22.397   Durbin-Watson:                   1.367
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               27.888
Skew:                           0.282   Prob(JB):                     8.79e-07
Kurtosis:                       3.624   Cond. No.                     1.12e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.12e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [37]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=13)

In [55]:
from sklearn.preprocessing import PolynomialFeatures
polynomial_converter=PolynomialFeatures(degree=3)
X_train_poly=pd.DataFrame(polynomial_converter.fit_transform(X_train))

from sklearn.linear_model import LinearRegression
model=LinearRegression()
model.fit(X_train_poly,y_train)

ypred_train=model.predict(X_train_poly)

from sklearn.metrics import mean_squared_error,r2_score
print("Train RMSE:",np.sqrt(mean_squared_error(y_train,ypred_train)))
print("Train R2:",r2_score(y_train,ypred_train))

from sklearn.model_selection import cross_val_score
print("Cross Validation Score:",cross_val_score(model,X_train_poly,y_train,cv=5).mean())


Train RMSE: 3.9180284998519768
Train R2: 0.9429515502073185
Cross Validation Score: 0.7617502614622651


In [39]:
X_test_poly=pd.DataFrame(polynomial_converter.transform(X_test))

ypred_test=model.predict(X_test_poly)

print("Test RMSE:",np.sqrt(mean_squared_error(y_test,ypred_test)))
print("Test R2:",r2_score(y_test,ypred_test))

Test RMSE: 5.390042183781404
Test R2: 0.9002463037113465


In [40]:
train_r2=[]
test_r2=[]

for i in range(1,10):
    polynomial_converter=PolynomialFeatures(degree=i)
    X_train_poly=pd.DataFrame(polynomial_converter.fit_transform(X_train))

    model=LinearRegression()
    model.fit(X_train_poly,y_train)

    train_pred=model.predict(X_train_poly)
    train_r2.append(model.score(X_train_poly,y_train))

    X_test_poly=pd.DataFrame(polynomial_converter.transform(X_test))

    test_pred=model.predict(X_test_poly)
    test_r2.append(model.score(X_test_poly,y_test))

In [41]:
train_r2

[0.8427085567164938,
 0.8922344924689057,
 0.9515239287106124,
 0.9873258199148023,
 0.9946918465982063,
 0.9957763663812926,
 0.9958335398049105,
 0.9959479796484352,
 0.9960601460050298]

In [42]:
test_r2

[0.8374482414097246,
 0.9002463037113465,
 0.9253474402304492,
 -7.1608616905453655,
 -1062442134.0591986,
 -6987697.3278168505,
 -5710869812.943979,
 -2291445151.5650816,
 -247863438.97443745]

In [43]:
final_poly_converter=PolynomialFeatures(degree=3)
X_train_poly=pd.DataFrame(final_poly_converter.fit_transform(X_train))

final_model=LinearRegression()
final_model.fit(X_train_poly,y_train)

train_pred=final_model.predict(X_train_poly)
print("train R2:",final_model.score(X_train_poly,y_train))
print("Cross validation score:",cross_val_score(model,X_train_poly,y_train,cv=5).mean())

X_test_poly=pd.DataFrame(final_poly_converter.transform(X_test))
test_pred=final_model.predict(X_test_poly)
print("test R2:",final_model.score(X_test_poly,y_test))

train R2: 0.9515239287106124
Cross validation score: 0.8156849469292033
test R2: 0.9253474402304492


In [44]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=9)

In [45]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
estimator=Lasso()
param_grid={"alpha":list(range(1,100))}

model_hp=GridSearchCV(estimator,param_grid,cv=5,scoring='r2')

model_hp.fit(X_train,y_train)
model_hp.best_params_

{'alpha': 1}

In [53]:
from sklearn.linear_model import Lasso
lasso_best=Lasso(alpha=1)
lasso_best.fit(X_train,y_train)

print("Intercept:",lasso_best.intercept_)
print("coefficients:",lasso_best.coef_)

ypred_train=lasso_best.predict(X_train)
from sklearn.metrics import r2_score
print("train r2:",r2_score(y_train,ypred_train))
from sklearn.model_selection import cross_val_score
print("Cv score:",cross_val_score(lasso_best,X_train,y_train,cv=5).mean())

ypred_test=lasso_best.predict(X_test)
print("test r2:",r2_score(y_test,ypred_test))


Intercept: -19.92101695694874
coefficients: [ 0.12033679  0.09534064  0.06978623 -0.18209534  0.08046823  0.00897835
  0.01677401  4.5667302 ]
train r2: 0.841540468647293
Cv score: 0.836759244881198
test r2: 0.8374494560671442


In [47]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
estimator=Ridge()

param_grid={"alpha":list(range(1,100))}

model_hp=GridSearchCV(estimator,param_grid,cv=5,scoring='r2')

model_hp.fit(X_train,y_train)
model_hp.best_params_

{'alpha': 1}

In [52]:
ridge_best=Ridge(alpha=1)
ridge_best.fit(X_train,y_train)

print("intercept:",ridge_best.intercept_)
print("coefficients:",ridge_best.coef_)

ypred_train=ridge_best.predict(X_train)
print("train r2:",r2_score(y_train,ypred_train))
print("cv Score:",cross_val_score(ridge_best,X_train,y_train,cv=5).mean())

ypred_test=ridge_best.predict(X_test)
print("test r2:",r2_score(y_test,ypred_test))

intercept: -33.08046786741762
coefficients: [ 0.12257795  0.09784867  0.07072289 -0.16185852  0.15105698  0.01329299
  0.02062377  4.78476501]
train r2: 0.8425946479731455
cv Score: 0.8376928235307005
test r2: 0.8380652735739494


In [49]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=9)

from sklearn.linear_model import ElasticNet
enr_base=ElasticNet()
enr_base.fit(X_train,y_train)

train_predictions=enr_base.predict(X_train)
test_predictions=enr_base.predict(X_test)

print("train R2:",enr_base.score(X_train,y_train))
print("test R2:",enr_base.score(X_test,y_test))
from sklearn.model_selection import cross_val_score
print("cross Validation Score:",cross_val_score(enr_base,X,y,cv=5).mean())

train R2: 0.8368372338439635
test R2: 0.8301046190430988
cross Validation Score: 0.7420314192350503


In [50]:
from sklearn.model_selection import GridSearchCV

estimator=ElasticNet()

param_grid={"alpha":[0.1,0.2,1,2,3,5,10],"l1_ratio":[0.1,0.5,0.75,0.9,0.95,1]}

model_hp=GridSearchCV(estimator,param_grid,cv=5,scoring='neg_mean_squared_error')
model_hp.fit(X_train,y_train)
model_hp.best_params_

{'alpha': 0.1, 'l1_ratio': 1}

In [51]:
enr_best=ElasticNet(alpha=5,l1_ratio=0.1)
enr_best.fit(X_train,y_train)

print("intercept:",enr_best.intercept_)
print("coefficients",enr_best.coef_)

train_predictions=enr_best.predict(X_train)
test_predictions=enr_best.predict(X_test)

print("Train R2:",enr_best.score(X_train,y_train))
print("test R2:",enr_best.score(X_test,y_test))
print("cross validation score:",cross_val_score(enr_best,X,y,cv=5).mean())

intercept: -9.68244204616984
coefficients [ 0.1164674   0.09233206  0.07002229 -0.16125173  0.20456861  0.00808886
  0.01395571  2.29531793]
Train R2: 0.7425634218268891
test R2: 0.7412009884410145
cross validation score: 0.6344494202086242
